Author: Brian Erichsen Fagundes

Data Manipulation and NN architecture

In [5]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

#data = pd.read_csv('stock_data.csv')
# fetches data from excel file
excel_data = pd.read_excel('historical_prices.xlsx', sheet_name=None)
combined_data = pd.DataFrame() # initiate empty data that will be combined

# per each sheet in excell file set stock to specific stock and concatenate data
for sheet_name, df in excel_data.items():
    df['Stock'] = sheet_name
    combined_data = pd.concat([combined_data, df], ignore_index=True)

#print(combined_data.info())
#print(combined_data.columns)
#print(combined_data.head())
#print(combined_data.tail())

# this step is crucial for NN LSTM training
scaler = MinMaxScaler(feature_range=(0, 1))

scaled_data = scaler.fit_transform(combined_data[['Close', 'Volume']])

sequence_length = 60
x_train, y_train = [],[]
# splits x_t and y_t
# Len of data number of training examples, sequence len is number of steps, 1 feature
# For y_train - 1D array; since we keeping only stock prices (1 feature)
for i in range(sequence_length, len(scaled_data)):
    x_train.append(scaled_data[i-sequence_length:i, 0])
    y_train.append(scaled_data[i, 0])

# transforms data into numpy array for model
x_train, y_train = np.array(x_train), np.array(y_train)
# reshapes it to 3D (samples, time steps, features)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

# model.compile - learn process before training the model
# adam - adaptive learning - and MSE for regression
model.compile(optimizer='adam', loss='mean_squared_error')
# trains the model in provided data -- x input; y target (labels)
#model.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2, shuffle=True)
model.fit(x_train, y_train, batch_size=1, epochs=1)

model.save('stock_prediction_model.h5')

/var/folders/vz/hr6jrn1n0ggf1mxbfs500y4m0000gn/T/ipykernel_14101/3806401752.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat([combined_data, df], ignore_index=True)
/Users/brianerichsenfagundes/Desktop/6017/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2576/2576 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - loss: 0.0062


In [7]:
from sklearn.model_selection import train_test_split

#splitting the data into train and test sets
sequence_length = 60
x_data, y_data = [], []

for i in range(sequence_length, len(scaled_data)):
    x_data.append(scaled_data[i-sequence_length:i, 0])
    y_data.append(scaled_data[i, 0])

x_data, y_data = np.array(x_data), np.array(y_data)
train_size = int(len(x_data) * 0.8)
#[x:y] -- from 0 to train size and from train size to the end
x_train, x_test = x_data[:train_size], x_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

#reshape data for LSTM model which expects a 3D
# input - samples, time steps and number of features
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [8]:
# adam - adaptive learning - and MSE for regression
model.compile(optimizer='adam', loss='mean_squared_error')
# trains the model in provided data -- x input; y target (labels)
model.fit(x_train, y_train, batch_size=1, epochs=1)

2060/2060 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.0028


In [9]:
from sklearn.metrics import mean_squared_error
# Model Evaluation
predicted_stock_price = model.predict(x_test)
# Mean squared error calculation
mse = mean_squared_error(y_test, predicted_stock_price)
sqmse = np.sqrt(mse)
print(f'Test MSE: {mse:.4f}')
print(f'Test SQMSE: {sqmse:.4f}')

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Test MSE: 0.0017
Test SQMSE: 0.0414


So far it seems that model can predictict well the prices of stocks